Используем код Task3 для проверки своей реализации сверточного слоя.

Реализация через циклы очень неэффективна по производительности. Есть целых два способа сделать то же самое с помощью матричного умножения. 

На этом шаге будет реализация первым из них.

 

Рассмотрим свертку одного одноканального изображения размером 4*4 пикселя (значения пикселей обозначены через X).

Сворачивать будем с ядром из одного фильтра размером 3*3, веса обозначены через W.

Для простоты примем stride = 1.

Тогда выход Y будет иметь размерность 1*1*2*2 (в данном случае на входе одно изображение - это первая единица в размерности, в ядре один фильтр - это вторая единица в размерности выхода).

<img src="img\conv51.png" width="50%">

Оказывается, выход свертки можно получить умножением матриц, как показано ниже.

<img src="img\conv52.png" width="50%">


Рекомендуем убедиться в этом, перемножив матрицы на листочке.

Давайте перейдем от простого случая к общему:

    Если фильтров в ядре больше одного. Заметим, что для каждого фильтра, матрица W’ будет умножаться на один и тот же вектор изображения. Значит, можно сконкатенировать матрицы фильтров ядра по вертикали и за одно умножение получить ответ для всех фильтров.

 

<img src="img\conv53.svg" width="50%">
 

    Если на входе более одного изображения: заметим, что матрица W’ одинакова для всех изображений батча, то есть, можно каждое изображение вначале вытянуть в столбец, а затем эти столбцы для всех изображений батча сконкатенировать по горизонтали.

 <img src="img\conv54.svg" width="50%">
 

    Если в изображении больше одного слоя, вначале выполним преобразования входа и ядра для каждого слоя, а затем сконкатенируем: вектора разных слоев входа в один большой вектор, а матрицы ядра соответственно в одну длинную матрицу. И мы получим сложение от выходов по слоям в процессе перемножения матриц.

 <img src="img\conv55.png" width="50%">

То есть даже в самом общем случае мы за одно умножение матриц можем получить ответ.

Но рассчитанный таким способом выход не совпадает по размерности с выходом стандартного слоя из PyTorch - нужно изменить размерность.

 

В коде уже реализовано:

    преобразование входного батча изображений

    умножение матрицы ядра на матрицу входа

    преобразование ответа

Напоминание: во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.

Вам осталось реализовать преобразование ядра в описанный выше формат.

Обратите внимание, что в коде рассматривается общий случай - вход состоит из нескольких многослойных изображений, в ядре несколько слоев.

In [1]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        output_tensor_shape = calc_out_shape(input_matrix_shape=torch_input.shape, out_channels=self.kernel.shape[0], kernel_size=self.kernel.shape[2], stride=self.stride, padding=0)

        in_img_channel_num = torch_input.shape[1]
        in_img_height = torch_input.shape[2]
        in_img_width = torch_input.shape[3]
        in_img_length = in_img_height * in_img_width

        filter_num = output_tensor_shape[1]
        filter_height = self.kernel.shape[2]
        filter_width = self.kernel.shape[2]
        
        N = in_img_length * in_img_channel_num
        M = output_height * output_width * filter_num
        kernel_unsqueezed = torch.zeros([M, N])

        for f in range(filter_num):
            for m in range(output_height):
                for n in range(output_width):
                    for c in range(in_img_channel_num):
                        for row in range(filter_height):
                            n_shift = c * in_img_length + row * in_img_width +\
                                      self.stride * n + self.stride * m * in_img_width
                            m_shift = f * output_height * output_width + m * output_width + n
                            kernel_unsqueezed[m_shift, n_shift : n_shift+filter_width] = self.kernel[f, c, row]
        return kernel_unsqueezed

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))



In [2]:
# Проверка происходит автоматически вызовом следующего кода
print(test_conv2d_layer(Conv2dMatrix))

True
